In [4]:
#import libraries
#!pip install kaggle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil



import kaggle

#download dataset using kaggle api
kaggle.api.authenticate()

kaggle.api.dataset_download_files('ankitbansal06/retail-orders', path='.', unzip=True)


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [90]:
#load csv into pandas df and handling null values
df = pd.read_csv('orders.csv', na_values = ['Not Available', 'unknown'])
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [91]:
#standardizing columns: lower-cased, separated by _
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [92]:
#deriving new columns for discount, sale price, and profit

df['discount'] = df['list_price']*df['discount_percent']*0.01
df['sale_price'] = df['list_price'] - df['discount'] #price after discount
df['profit'] = df['sale_price'] - df['cost_price']

In [93]:
#converting order_date to datetime format
df['order_date'] = pd.to_datetime(df['order_date'], format = '%Y-%m-%d')

In [94]:
#dropping columns: list_price and discount_percent as they wont be used going forward
df = df.drop(['list_price', 'cost_price', 'discount_percent'], axis = 1)

In [95]:
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0


In [16]:
#load data into sql server
#!pip install sqlalchemy
#!pip install psycopg2-binary
from sqlalchemy import create_engine
import psycopg2

# Connection string (replace with your actual values)
host = 'localhost'
port = 5432
dbname = 'postgres'
user = 'postgres'
password = '1234'

# Create a connection string for SQLAlchemy
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Create SQLAlchemy engine
engine = create_engine(conn_str)

# Test the connection
with engine.connect() as connection:
    result = connection.execute('SELECT version();')
    print(f"Connected to PostgreSQL, version: {result.fetchone()[0]}")

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/61/69/3b3d7bd583c6d3cbe5100802efa5beacaacc86e37b653fc708bf3d6853b8/psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 87.5 kB/s eta 0:00:13
   - -------------------------------------- 0.0/1.2 MB 87.5 kB/s eta

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\PMYLS\\AppData\\Roaming\\Python\\Python311\\site-packages\\psycopg2\\_psycopg.cp311-win_amd64.pyd'
Check the permissions.



In [115]:
#load data into postgres server
with engine.connect() as conn:
    df.to_sql('df_orders', con=conn, index=False, if_exists='replace')

In [116]:
# Write a SQL query (replace with your actual query)
query = 'SELECT * FROM df_orders LIMIT 10;'  # Example query to fetch first 10 rows from df_orders

# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit,year,month
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8,2023,3
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1,2023,8
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5,2023,1
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8,2022,6
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0,2022,7
5,6,2022-03-13,None,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,7,1.5,48.5,-1.5,2022,3
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,4,0.3,9.7,-0.3,2022,12
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,6,45.5,864.5,4.5,2022,1
8,9,2023-03-23,None,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,3,0.4,19.6,-0.4,2023,3
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,5,3.3,106.7,16.7,2023,5


In [33]:
# Find top 10 highest revenue generating products

query = """SELECT product_id, sum(sale_price) as sales 
           FROM df_orders
           GROUP BY product_id
           ORDER BY sales desc
           LIMIT 10
"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result

,product_id,sales
0,TEC-CO-10004722,59514.0
1,OFF-BI-10003527,26525.3
2,TEC-MA-10002412,21734.4
3,FUR-CH-10002024,21096.2
4,OFF-BI-10001359,19090.2
5,OFF-BI-10000545,18249.0
6,TEC-CO-10001449,18151.2
7,TEC-MA-10001127,17906.4
8,OFF-BI-10004995,17354.8
9,OFF-SU-10000151,16325.8


In [80]:
#find top 5 highest selling products in each region

query = """WITH ranked_products AS (
    SELECT
        region,
        product_id,
        SUM(sale_price) AS total_sales,
        ROW_NUMBER() OVER (PARTITION BY region ORDER BY SUM(sale_price) DESC) AS rank
    FROM
        df_orders
    GROUP BY
        region, product_id
)
SELECT
    region,
    product_id,
    total_sales
FROM
    ranked_products
WHERE
    rank <= 5
ORDER BY
    region,
    rank

"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result

,region,product_id,total_sales
0,Central,TEC-CO-10004722,16975.0
1,Central,TEC-MA-10000822,13770.0
2,Central,OFF-BI-10001120,11056.5
3,Central,OFF-BI-10000545,10132.7
4,Central,OFF-BI-10004995,8416.1
5,East,TEC-CO-10004722,29099.0
6,East,TEC-MA-10001047,13767.0
7,East,FUR-BO-10004834,11274.1
8,East,OFF-BI-10001359,8463.6
9,East,TEC-CO-10001449,8316.0


In [102]:
#find month over month growth comparison for 2022 and 2023, jan 2022 vs jan 2023
#df['order_date'] = pd.to_datetime(df['order_date'])
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month
df.head()

df2022 = df[df['year']==2022].groupby(['month'])['sale_price'].sum().reset_index()
df2023 = df[df['year']==2023].groupby(['month'])['sale_price'].sum().reset_index()


In [113]:
df2023['sale_price_2022'] = df2022['sale_price']
df2023['change'] = 100*(df2023['sale_price']-df2023['sale_price_2022'])/df2023['sale_price_2022']
df2023

,month,sale_price,sale_price_2022,change
0,1,88632.6,94712.5,-6.419322
1,2,128124.2,90091.0,42.216426
2,3,82512.3,80106.0,3.003895
3,4,111568.6,95451.6,16.884997
4,5,86447.9,79448.3,8.810258
5,6,68976.5,94170.5,-26.753601
6,7,90563.8,78652.2,15.144649
7,8,87733.6,104808.0,-16.291123
8,9,76658.6,79142.2,-3.138149
9,10,121061.5,118912.7,1.807040


In [144]:
#find month over month growth comparison for 2022 and 2023, jan 2022 vs jan 2023


query = """
    SELECT 
        EXTRACT(YEAR FROM order_date) AS year,
        EXTRACT(MONTH FROM order_date) AS month,
        SUM(sale_price) AS total_sales
    FROM 
        df_orders
    WHERE
        EXTRACT(YEAR FROM order_date) = 2023
    GROUP BY 
        EXTRACT(YEAR FROM order_date), EXTRACT(MONTH FROM order_date)

"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result


,year,month,total_sales
0,2023.0,1.0,88632.6
1,2023.0,2.0,128124.2
2,2023.0,3.0,82512.3
3,2023.0,4.0,111568.6
4,2023.0,5.0,86447.9
5,2023.0,6.0,68976.5
6,2023.0,7.0,90563.8
7,2023.0,8.0,87733.6
8,2023.0,9.0,76658.6
9,2023.0,10.0,121061.5


In [177]:
#find month over month growth comparison for 2022 and 2023, jan 2022 vs jan 2023


query = """
SELECT 
    a.month,
    a.total_sales AS total_sales_2022,
    b.total_sales AS total_sales_2023,
    CASE
        WHEN b.total_sales IS NULL THEN NULL
        ELSE (b.total_sales - a.total_sales) / a.total_sales * 100
    END AS month_over_month_growth
FROM 
    (SELECT 
        EXTRACT(MONTH FROM order_date) AS month,
        SUM(sale_price) AS total_sales
    FROM 
        df_orders
    WHERE 
        EXTRACT(YEAR FROM order_date) = 2022
    GROUP BY 
        EXTRACT(MONTH FROM order_date)
    ) a
LEFT JOIN 
    (SELECT 
        EXTRACT(MONTH FROM order_date) AS month,
        SUM(sale_price) AS total_sales
    FROM 
        df_orders
    WHERE 
        EXTRACT(YEAR FROM order_date) = 2023
    GROUP BY 
        EXTRACT(MONTH FROM order_date)
    ) b
ON 
    a.month = b.month
ORDER BY 
    a.month;


        
"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result


,month,total_sales_2022,total_sales_2023,month_over_month_growth
0,1.0,94712.5,88632.6,-6.419322
1,2.0,90091.0,128124.2,42.216426
2,3.0,80106.0,82512.3,3.003895
3,4.0,95451.6,111568.6,16.884997
4,5.0,79448.3,86447.9,8.810258
5,6.0,94170.5,68976.5,-26.753601
6,7.0,78652.2,90563.8,15.144649
7,8.0,104808.0,87733.6,-16.291123
8,9.0,79142.2,76658.6,-3.138149
9,10.0,118912.7,121061.5,1.807040


In [146]:
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit,year,month
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8,2023,3
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1,2023,8
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5,2023,1
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8,2022,6
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0,2022,7


In [183]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit', 'year',
       'month'],
      dtype='object')

In [165]:
#for each category, which month had highest_sales

query = """
SELECT a.category, a.year_month
FROM (
    SELECT 
        category, 
        EXTRACT(YEAR FROM order_date) * 100 + EXTRACT(MONTH FROM order_date) AS year_month,  -- Concatenate year and month
        SUM(sale_price) AS sales,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY SUM(sale_price) DESC) AS rank
    FROM 
        df_orders
    GROUP BY 
        category, EXTRACT(YEAR FROM order_date) * 100 + EXTRACT(MONTH FROM order_date)
) a
WHERE a.rank = 1
"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result

,category,year_month
0,Furniture,202210.0
1,Office Supplies,202302.0
2,Technology,202310.0


In [166]:
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit,year,month
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8,2023,3
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1,2023,8
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5,2023,1
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8,2022,6
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0,2022,7


In [182]:
#which subcategory had highest growth by profit in 2023 compared to 2022

query = """
SELECT a.sub_category, a.profit as profit_2022, b.profit as profit_2023,
(b.profit - a.profit) / a.profit * 100 as profit_growth
FROM
(SELECT sub_category, extract(year from order_date) as year, sum(profit) as profit
FROM df_orders
WHERE extract(year from order_date) = 2022
GROUP BY sub_category, extract(year from order_date)) a
LEFT JOIN
(SELECT sub_category, extract(year from order_date) as year, sum(profit) as profit
FROM df_orders
WHERE extract(year from order_date) = 2023
GROUP BY sub_category, extract(year from order_date)) b
ON a.sub_category = b.sub_category
ORDER BY profit_growth desc
LIMIT 1
"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result

,sub_category,profit_2022,profit_2023,profit_growth
0,Machines,7243.2,10878.5,50.189143


In [77]:
#find top 5 highest selling products in each region

query = """
    WITH RANKED_PRODUCTS AS(
    SELECT
        region,
        product_id,
        SUM(sale_price) AS sales
    FROM
        df_orders
    GROUP BY
        region, product_id
    ORDER BY
        region, sales desc
        ),
    RANKED_PRODUCTS_WITH_RANK AS(
    SELECT *, row_number() over (partition by region order by sales desc) AS RANK
    FROM RANKED_PRODUCTS
    )
    SELECT 
    region,
    product_id,
    sales,
    RANK
FROM 
    RANKED_PRODUCTS_WITH_RANK
WHERE 
    RANK <= 5
"""
# Execute the query and load the result into a pandas DataFrame
df_result = pd.read_sql(query, con=engine)
df_result

,region,product_id,sales,rank
0,Central,TEC-CO-10004722,16975.0,1
1,Central,TEC-MA-10000822,13770.0,2
2,Central,OFF-BI-10001120,11056.5,3
3,Central,OFF-BI-10000545,10132.7,4
4,Central,OFF-BI-10004995,8416.1,5
5,East,TEC-CO-10004722,29099.0,1
6,East,TEC-MA-10001047,13767.0,2
7,East,FUR-BO-10004834,11274.1,3
8,East,OFF-BI-10001359,8463.6,4
9,East,TEC-CO-10001449,8316.0,5


In [54]:
df_grouped = df.groupby(['region', 'product_id'], as_index=False)['sale_price'].sum()
df_sorted = df_grouped.sort_values(by=['region', 'sale_price'], ascending=[True, False])
df_top5 = df_sorted.groupby('region').head(5)

df_top5


,region,product_id,sale_price
1166,Central,TEC-CO-10004722,16975.0
1168,Central,TEC-MA-10000822,13770.0
488,Central,OFF-BI-10001120,11056.5
469,Central,OFF-BI-10000545,10132.7
617,Central,OFF-BI-10004995,8416.1
2556,East,TEC-CO-10004722,29099.0
2565,East,TEC-MA-10001047,13767.0
1342,East,FUR-BO-10004834,11274.1
1834,East,OFF-BI-10001359,8463.6
2548,East,TEC-CO-10001449,8316.0


,region,product_id,sale_price
1166,Central,TEC-CO-10004722,16975.0
1168,Central,TEC-MA-10000822,13770.0
488,Central,OFF-BI-10001120,11056.5
469,Central,OFF-BI-10000545,10132.7
617,Central,OFF-BI-10004995,8416.1
2556,East,TEC-CO-10004722,29099.0
2565,East,TEC-MA-10001047,13767.0
1342,East,FUR-BO-10004834,11274.1
1834,East,OFF-BI-10001359,8463.6
2548,East,TEC-CO-10001449,8316.0


In [2]:
pip install streamlit


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/3d/b0/031933cb253d757dfc8791707d413ce90ab52ec7fb4524ef955070136d31/streamlit-1.43.2-py2.py3-none-any.whl.metadata
  Using cached streamlit-1.43.2-py2.py3-none-any.whl.metadata (8.9 kB)
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Setup the database connection (use your actual connection details)
# Connection string (replace with your actual values)
host = 'localhost'
port = 5432
dbname = 'postgres'
user = 'postgres'
password = '1234'

# Create a connection string for SQLAlchemy
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Create SQLAlchemy engine
engine = create_engine(conn_str)

# Function to load data from SQL
def load_data(query):
    return pd.read_sql(query, con=engine)

# Streamlit dashboard layout
st.title('Sales Dashboard')

# Sidebar for navigation
st.sidebar.title('Navigation')
options = ['Top 10 Revenue Generating Products', 'Top 5 Highest Selling Products by Region', 
           'Month Over Month Growth Comparison', 'Highest Sales Month by Category', 
           'Subcategory Growth by Profit in 2023 vs 2022']

choice = st.sidebar.selectbox('Select a question to view:', options)

# Question 1: Top 10 highest revenue-generating products
if choice == 'Top 10 Revenue Generating Products':
    query = """
    SELECT product_id, SUM(sale_price) AS total_revenue
    FROM df_orders
    GROUP BY product_id
    ORDER BY total_revenue DESC
    LIMIT 10;
    """
    df = load_data(query)
    st.subheader('Top 10 Revenue Generating Products')
    st.dataframe(df)

    # Plotting the revenue of top 10 products
    plt.figure(figsize=(10, 6))
    sns.barplot(x='product_id', y='total_revenue', data=df)
    plt.title('Top 10 Revenue Generating Products')
    plt.xticks(rotation=45)
    st.pyplot()

# Question 2: Top 5 highest-selling products in each region
if choice == 'Top 5 Highest Selling Products by Region':
    query = """
    SELECT region, product_id, SUM(quantity) AS total_sales
    FROM df_orders
    GROUP BY region, product_id
    ORDER BY region, total_sales DESC;
    """
    df = load_data(query)
    st.subheader('Top 5 Highest Selling Products in Each Region')

    # Display top 5 per region
    df_region = df.groupby('region').head(5)  # Top 5 per region
    st.dataframe(df_region)

    # Plotting the top-selling products
    plt.figure(figsize=(10, 6))
    sns.barplot(x='product_id', y='total_sales', hue='region', data=df_region)
    plt.title('Top 5 Highest Selling Products in Each Region')
    plt.xticks(rotation=45)
    st.pyplot()

# Question 3: Month-over-month growth comparison for 2022 and 2023
if choice == 'Month Over Month Growth Comparison':
    query = """
    WITH monthly_sales AS (
        SELECT EXTRACT(YEAR FROM order_date) AS year,
               EXTRACT(MONTH FROM order_date) AS month,
               SUM(sale_price) AS total_sales
        FROM df_orders
        WHERE EXTRACT(YEAR FROM order_date) IN (2022, 2023)
        GROUP BY year, month
    )
    SELECT year, month, total_sales,
           LAG(total_sales) OVER (PARTITION BY year ORDER BY month) AS previous_month_sales
    FROM monthly_sales;
    """
    df = load_data(query)
    df['growth'] = (df['total_sales'] - df['previous_month_sales']) / df['previous_month_sales'] * 100

    st.subheader('Month Over Month Growth Comparison for 2022 and 2023')
    st.dataframe(df)

    # Plotting month-over-month growth
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='month', y='growth', hue='year', data=df)
    plt.title('Month-over-Month Growth Comparison for 2022 and 2023')
    st.pyplot()

# Question 4: Highest sales month by category
if choice == 'Highest Sales Month by Category':
    query = """
    SELECT category, EXTRACT(MONTH FROM order_date) AS month, SUM(sale_price) AS total_sales
    FROM df_orders
    GROUP BY category, month
    ORDER BY category, total_sales DESC;
    """
    df = load_data(query)
    df_category_max_sales = df.groupby('category').head(1)  # Getting the month with highest sales for each category

    st.subheader('Highest Sales Month by Category')
    st.dataframe(df_category_max_sales)

    # Plotting
    plt.figure(figsize=(10, 6))
    sns.barplot(x='category', y='total_sales', data=df_category_max_sales)
    plt.title('Highest Sales Month by Category')
    st.pyplot()

# Question 5: Subcategory growth by profit in 2023 vs 2022
if choice == 'Subcategory Growth by Profit in 2023 vs 2022':
    query = """
    SELECT sub_category, EXTRACT(YEAR FROM order_date) AS year, SUM(profit) AS total_profit
    FROM df_orders
    WHERE EXTRACT(YEAR FROM order_date) IN (2022, 2023)
    GROUP BY sub_category, year
    ORDER BY sub_category, year;
    """
    df = load_data(query)

    # Pivoting the data to have 2022 and 2023 profit comparison
    df_pivot = df.pivot(index='sub_category', columns='year', values='total_profit')
    df_pivot['growth'] = (df_pivot[2023] - df_pivot[2022]) / df_pivot[2022] * 100

    st.subheader('Subcategory Growth by Profit in 2023 vs 2022')
    st.dataframe(df_pivot)

    # Plotting profit growth by subcategory
    plt.figure(figsize=(10, 6))
    sns.barplot(x=df_pivot.index, y='growth', data=df_pivot)
    plt.title('Subcategory Growth by Profit in 2023 vs 2022')
    st.pyplot()


2025-03-19 12:23:35.949 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 12:23:36.164 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-19 12:23:36.164 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 12:23:36.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 12:23:36.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 12:23:36.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 12:23:36.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 12:23:36.168 Thread 'MainThread': mi